In [2]:
import omdb
import sqlite3
import pandas as pd
import os
import requests
import shutil
import io
import PIL.Image as Image

# first read in OMDB python key
with open('api_key') as f:
    lines = f.readlines()
api_key = lines[0]
omdb.set_default('apikey', api_key)


In [3]:
# set acronyms
st = {'tos': {'title': 'The Original Series', 'imdbid': 'tt0060028'},
      'tas': {'title': 'The Animated Series', 'imdbid': 'tt0069637'},
      'tng': {'title': 'The Next Generation', 'imdbid': 'tt0092455'},
      'ds9': {'title': 'Deep Space Nine',     'imdbid': 'tt0106145'},
      'voy': {'title': 'Voyager',             'imdbid': 'tt0112178'},
      'ent': {'title': 'Enterprise',          'imdbid': 'tt0244365'},
      'dis': {'title': 'Discovery',           'imdbid': 'tt5171438'},
      'sho': {'title': 'Short Treks',         'imdbid': 'tt9059594'},
      'pic': {'title': 'Picard',              'imdbid': 'tt8806524'},
      'lds': {'title': 'Lower Decks',         'imdbid': 'tt9184820'},
      'pro': {'title': 'Prodigy',             'imdbid': 'tt9795876'},
      'snw': {'title': 'Strange New Worlds',  'imdbid': 'tt12327578'},
      } 

## Series Request

In [6]:
db_filename = "treklist.db"

# delete current database
os.remove(db_filename)

# make omdb requests for series
conn = sqlite3.connect(db_filename)
curs = conn.cursor()

conn.execute("DROP TABLE IF EXISTS series")
conn.execute("CREATE TABLE series (abb TEXT, title TEXT, imdb_id TEXT, year TEXT, total_seasons INTEGER, poster_url TEXT, poster BLOB, rated TEXT)")
for key in st.keys():
    res = omdb.imdbid(st[key]['imdbid'])
    img = requests.get(res['poster'], stream=True)

    cmd = "INSERT INTO series (abb, title, imdb_id, year, total_seasons, poster_url, rated) VALUES (?, ?, ?, ?, ?, ?, ?) "
         #f"('{key}', '{}', '{}', '{}', {}, ?, '{res['rated']}');"
    curs.execute(cmd, [key, res['title'], res['imdb_id'], res['year'], res['total_seasons'], res['poster'], res['rated']])
    conn.commit()

    
df = pd.read_sql_query("SELECT * FROM series", conn)
print(df)



    abb                           title     imdb_id       year  total_seasons  \
0   tos                       Star Trek   tt0060028  1966–1969              3   
1   tas  Star Trek: The Animated Series   tt0069637  1973–1975              2   
2   tng  Star Trek: The Next Generation   tt0092455  1987–1994              7   
3   ds9      Star Trek: Deep Space Nine   tt0106145  1993–1999              7   
4   voy              Star Trek: Voyager   tt0112178  1995–2001              7   
5   ent           Star Trek: Enterprise   tt0244365  2001–2005              4   
6   dis            Star Trek: Discovery   tt5171438      2017–              5   
7   sho          Star Trek: Short Treks   tt9059594      2018–              2   
8   pic               Star Trek: Picard   tt8806524      2020–              3   
9   lds          Star Trek: Lower Decks   tt9184820      2020–              3   
10  pro              Star Trek: Prodigy   tt9795876      2021–              1   
11  snw   Star Trek: Strange

## Episodes Request

In [23]:
keys = list(st.keys())
#keys = keys[0:2] # temporary to keep episode api req low
for key in keys: # temporary for testing
#key = keys[8]
#conn.execute(f"DROP TABLE IF EXISTS {key}")
    conn.execute(f"CREATE TABLE IF NOT EXISTS {key} (title TEXT, rated TEXT, released DATE, season INTEGER, episode INTEGER, runtime TEXT, director TEXT, writer TEXT, actors TEXT, plot TEXT, poster_url TEXT, poster BLOB, imdb_rating FLOAT, imdb_votes INTEGER, imdb_id TEXT)")

    tot_seasons = df['total_seasons'][df['abb'] == key].values[0]
    print('key %s seasons %i' % (key, tot_seasons))
    for season in range(tot_seasons):# another temporary....
        season += 1 # account for zero index
        res = omdb.imdbid(st[key]['imdbid'], season=season)
        for ep in res['episodes']:

            # check if it is already in the database
            df_check = pd.read_sql_query(f"SELECT * FROM {key} WHERE imdb_id = '{ep['imdb_id']}'", conn)
            ep_exists = True if len(df_check) > 0 else False
            if not ep_exists:
                ep_info = omdb.imdbid(ep['imdb_id'])
                cmd = f"INSERT INTO {key} (title, rated, released, season, episode, runtime, director, writer, actors, plot, poster_url, imdb_rating, imdb_votes, imdb_id) VALUES " + "(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
                curs.execute(cmd, [ep_info['title'], ep_info['rated'], ep['released'], season, ep_info['episode'], ep_info['runtime'], ep_info['director'], ep_info['writer'], ep_info['actors'], ep_info['plot'], ep_info['poster'], ep_info['imdb_rating'], ep_info['imdb_votes'], ep_info['imdb_id']])
    conn.commit()

key tos seasons 3
key tas seasons 2
key tng seasons 7
key ds9 seasons 7
key voy seasons 7
key ent seasons 4
key dis seasons 5
key sho seasons 2
key pic seasons 3
key lds seasons 3
key pro seasons 1
key snw seasons 1


In [11]:
list(st.keys())

['tos',
 'tas',
 'tng',
 'ds9',
 'voy',
 'ent',
 'dis',
 'sho',
 'pic',
 'lds',
 'pro',
 'snw']

In [7]:
res = omdb.imdbid("tt0094030")
res

{'title': 'Encounter at Farpoint',
 'year': '1987',
 'rated': 'TV-PG',
 'released': '26 Sep 1987',
 'season': '1',
 'episode': '1',
 'runtime': '92 min',
 'genre': 'Action, Adventure, Sci-Fi',
 'director': 'Corey Allen',
 'writer': 'Gene Roddenberry, D.C. Fontana, Tracy Tormé',
 'actors': 'Patrick Stewart, Jonathan Frakes, LeVar Burton',
 'plot': 'On the maiden mission of the U.S.S. Enterprise (NCC-1701-D), an omnipotent being known as Q challenges the crew to discover the secret of a mysterious base in an advanced and civilized fashion.',
 'language': 'English',
 'country': 'United States',
 'awards': 'N/A',
 'poster': 'https://m.media-amazon.com/images/M/MV5BYTI2NDVlZTYtZGZlZi00YTYzLTg5YzEtYzBkOGMwM2Y3ZTY4XkEyXkFqcGdeQXVyNDMyNDEyNA@@._V1_SX300.jpg',
 'ratings': [{'source': 'Internet Movie Database', 'value': '6.9/10'}],
 'metascore': 'N/A',
 'imdb_rating': '6.9',
 'imdb_votes': '6431',
 'imdb_id': 'tt0094030',
 'series_id': 'tt0092455',
 'type': 'episode',
 'response': 'True'}

In [10]:
res['episodes']

[{'title': 'Remembrance',
  'released': '2020-01-23',
  'episode': '1',
  'imdb_rating': '8.2',
  'imdb_id': 'tt9381924'},
 {'title': 'Maps and Legends',
  'released': '2020-01-30',
  'episode': '2',
  'imdb_rating': '7.4',
  'imdb_id': 'tt9420276'},
 {'title': 'The End Is the Beginning',
  'released': '2020-02-06',
  'episode': '3',
  'imdb_rating': '7.4',
  'imdb_id': 'tt9420278'},
 {'title': 'Absolute Candor',
  'released': '2020-02-13',
  'episode': '4',
  'imdb_rating': 'N/A',
  'imdb_id': 'tt9420280'},
 {'title': 'Stardust City Rag',
  'released': '2020-02-20',
  'episode': '5',
  'imdb_rating': 'N/A',
  'imdb_id': 'tt9420282'},
 {'title': 'The Impossible Box',
  'released': '2020-02-27',
  'episode': '6',
  'imdb_rating': '7.8',
  'imdb_id': 'tt9420284'},
 {'title': 'Nepenthe',
  'released': '2020-03-05',
  'episode': '7',
  'imdb_rating': '8.3',
  'imdb_id': 'tt9420288'},
 {'title': 'Broken Pieces',
  'released': '2020-03-12',
  'episode': '8',
  'imdb_rating': 'N/A',
  'imdb_i

In [31]:
df_check = pd.read_sql_query(f"SELECT * FROM snw", conn)
df_check

,title,rated,released,season,episode,runtime,director,writer,actors,plot,poster_url,poster,imdb_rating,imdb_votes,imdb_id
0,Episode #1.1,N/A,2022-05-05,1,1,N/A,Akiva Goldsman,"Gene Roddenberry, Akiva Goldsman, Alex Kurtzman","Rebecca Romijn, Anson Mount, Christina Chong",N/A,N/A,None,N/A,N/A,tt12330364


## Download/Insert Posters

In [45]:
df = pd.read_sql_query(f"SELECT * from series", conn)
df
for i, row in df.iterrows():
    poster_url = row['poster_url']
    if poster_url != "N/A":
        # insert poster
        img = requests.get(row['poster_url'], stream=True)
        cmd = f"UPDATE series SET poster = ? WHERE imdb_id = '{row['imdb_id']}'"
        #with open('test.jpg','wb') as f:
        #    shutil.copyfileobj(img.raw, f)
        curs.execute(cmd, [sqlite3.Binary(img.content)])
        conn.commit()

        
    

In [50]:
# optional - read back a poster
curs.execute("SELECT * FROM series WHERE imdb_id = 'tt0060028'")
record = curs.fetchall()
image = Image.open(io.BytesIO(record[0][6]))
image.show()

In [47]:
df = pd.read_sql_query("SELECT * FROM series", conn)
df
#requests.get(row['poster_url'], stream=True)
#print(row['poster_url'])

,abb,title,imdb_id,year,total_seasons,poster_url,poster,rated
0,tos,Star Trek,tt0060028,1966–1969,3,https://m.media-amazon.com/images/M/MV5BNDRkMT...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,TV-PG
1,tas,Star Trek: The Animated Series,tt0069637,1973–1975,2,https://m.media-amazon.com/images/M/MV5BMzI0Y2...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,TV-Y7
2,tng,Star Trek: The Next Generation,tt0092455,1987–1994,7,https://m.media-amazon.com/images/M/MV5BOWFhYj...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,TV-PG
3,ds9,Star Trek: Deep Space Nine,tt0106145,1993–1999,7,https://m.media-amazon.com/images/M/MV5BMDc3OG...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,TV-PG
4,voy,Star Trek: Voyager,tt0112178,1995–2001,7,https://m.media-amazon.com/images/M/MV5BYWIwMT...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,TV-PG
5,ent,Star Trek: Enterprise,tt0244365,2001–2005,4,https://m.media-amazon.com/images/M/MV5BODg3Zm...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,TV-PG
6,dis,Star Trek: Discovery,tt5171438,2017–,5,https://m.media-amazon.com/images/M/MV5BNjg1NT...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,TV-14
7,sho,Star Trek: Short Treks,tt9059594,2018–,2,https://m.media-amazon.com/images/M/MV5BNWY2NW...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,TV-PG
8,pic,Star Trek: Picard,tt8806524,2020–,3,https://m.media-amazon.com/images/M/MV5BMmMzNT...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,TV-MA
9,lds,Star Trek: Lower Decks,tt9184820,2020–,3,https://m.media-amazon.com/images/M/MV5BNjZkNG...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,TV-14
